In [ ]:
import os
import re
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense, Flatten, InputLayer
from sklearn.preprocessing import LabelEncoder
from tensorflow.python.keras import utils
import keras
import imageio 
from PIL import Image 

In [ ]:
# Reading the data
train = pd.read_csv('agedetectiontrain/train.csv')

# Image resizing of test data into single numpy array
temp = []
for img_name in train.ID:
    img_path = os.path.join('agedetectiontrain/Train', img_name)
    img = imageio.imread(img_path)
    img = np.array(Image.fromarray(img).resize((32, 32))).astype('float32')    
    temp.append(img)

train_x = np.stack(temp)

# Normalizing the images
train_x = train_x / 255.

# Encoding the categorical variable to numeric
lb = LabelEncoder()
train_y = lb.fit_transform(train.Class)
train_y = keras.utils.to_categorical(train_y)

In [ ]:
# Specifying all the parameters we will be using in our network
input_num_units = (32, 32, 3)
hidden_num_units = 500
output_num_units = 3

epochs = 100
batch_size = 128

In [ ]:
def models_with_different_activation_fn(list_of_activation_fn):    
    
    for i in range(len(list_of_activation_fn)):        
        # Defining the network
        model = Sequential([
          InputLayer(input_shape=input_num_units),
          Flatten(),
          Dense(units=hidden_num_units, activation=list_of_activation_fn[i]),
          Dense(units=output_num_units, activation='softmax'),
        ])
        model.compile(loss='categorical_crossentropy',
                  optimizer=keras.optimizers.Adam(),
                  metrics=['accuracy'])

        # Traning the model and writing log files for TensorBoard in distinct directories                
        logdir = r'activation\\' + list_of_activation_fn[i] # Each log file needs to be written in a distinct directory. (Mandatory)
        
        # Writing graph will take time. Hence, keeping it False.
        cb = keras.callbacks.TensorBoard(log_dir=logdir, write_graph=False)         
        print('Building model using '+ list_of_activation_fn[i] + ' activation function')
        
        history = model.fit(train_x, train_y, epochs=epochs, 
                           validation_split=0.2,
                           callbacks=[cb])
        print('Model built sucessfully.')
        print('')

In [ ]:
act = ['linear', 'sigmoid', 'tanh', 'relu', 'softmax']
models_with_different_activation_fn(act)

#### Run following command in Anaconda prompt after changing directory to the place which holds `activation`
tensorboard --logdir=activation  
In case, if above command doesn't open Tensorboard in the browser then change the port manually as shown:  
tensorboard --logdir=activation/ --host localhost --port 8088  
  
Use .* under tensorboard to visualize all 4 graphs in first section.  
You can also change `Horizontal Axis` from `Step` to `Relative` which describes relative time taken by each algorithm to finish.